In [34]:
import requests,bs4,re,datetime,smtplib,re
from dateutil.parser import parse

recEmail = input("Email address:").strip()
nameStr = input("TV Series:").strip()
nameArr = nameStr.split(",")                  #Array of name of TV Series

#    import mysql.connector
#    mydb = mysql.connector.connect( 
#      host="localhost",
#      user="yourusername",
#      passwd="yourpassword",
#      database="mydatabase"
#   )  
#   use this to enter the data into a mySQL database with appropriate userID,Password

#   mycursor = mydb.cursor()

#   sql = "INSERT INTO customers (name) VALUES (%s)"
#   mycursor.execute(sql, nameArr)

#   mydb.commit()


s = smtplib.SMTP('smtp.gmail.com', 587) 
s.starttls() 
s.login("sendersmailid@gmail.com","@12345abcd")   #Senders mail ID # created a new email-ID with the same name.The code will run with this email-ID.
messageArr = []
message = ""

for name in nameArr:
    try:
        text = name
        text = name.replace (" ", "+")
        link = "https://next-episode.net/site-search-"+ text + ".html"
        res = requests.get(link);
        soup = bs4.BeautifulSoup(res.text,'html.parser')
        div = soup.find("div", {"class": "list_item"})
        
        if div is not None:
            span = div.find("span",{"class": "headlinehref"})
            atag = span.find("a")
            hreflink = "https://next-episode.net"+atag['href']
            res = requests.get(hreflink);
            soup = bs4.BeautifulSoup(res.text,'html.parser')
        string1 = str(soup)
        
        if "Canceled/Ended" in string1 :                                    #case where all TV Series has ended.
            message = "The show has finished streaming all its episodes."
        elif "Running" in string1 or "Upcoming Season Premiere" in string1:   
            nextEpisodeDiv = soup.find("div", {"id": "next_episode"})
            nextStr = str(nextEpisodeDiv)
            date = nextStr[nextStr.find("Date")+16:nextStr.find("Date")+34].strip()
            dt = parse(date)
            message = "The next episode airs on " + dt.strftime('%Y-%d-%m') + "."
        else:
            googlelink = "https://www.google.com/search?q=imdb url "+name
            res = requests.get(googlelink);
            soup = bs4.BeautifulSoup(res.text,'html.parser')
            div = soup.find("div", {"class": "hJND5c"})
            cite = div.find("cite")
            unknown = "episodes"
            link = "https://www.imdb"+cite.contents[2];
            res = requests.get(link)
            soup = bs4.BeautifulSoup(res.text,'html.parser')
            div = soup.find("div", {"class": "seasons-and-year-nav"})
            divStr = str(div)
            cc = cite.contents[2][4:]+"episodes?year="
            yearV = divStr[divStr.find(cc)+31:divStr.find(cc)+35]
            year = int(yearV)
            currentyear = datetime.datetime.now().year
            if year <= currentyear:
                message = "No information is currently available."
            else:
                message = "The next Season begins in " +yearV + "."
        messageArr.append(message) 
    except:
        print("There was some error. Check name or internet connection")
        message = "There was some error :("
        messageArr.append(message)
message="\n"
for series,status in zip(nameArr,messageArr):
    message = message + "Tv series name: "+series+"\n"
    message = message + "Status: "+status+"\n\n"
s.sendmail("sendersmailid@gmail.com", recEmail, message)
s.quit()

Email address:jhaaditya1998@gmail.com
TV Series:suits,friends,big bang theory
The next episode airs on 2018-11-10.


(221, b'2.0.0 closing connection 72-v6sm55827273pfr.115 - gsmtp')